<a href="https://colab.research.google.com/github/sowmyakavali/Computer-Vision/blob/main/Display_bboxes_write_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Read bounding boxes from yolov5 format annotations and display them

#### Libraries

In [ ]:
import os
import cv2
import glob
from tqdm import tqdm

#### class to generate different colors for different classes

In [ ]:
class Colors:
    # Ultralytics color palette https://ultralytics.com/
    def __init__(self):
        # hex = matplotlib.colors.TABLEAU_COLORS.values()
        hex = ('FF3838', 'FF9D97', 'FF701F', 'FFB21D', 'CFD231', '48F90A', '92CC17', '3DDB86', '1A9334', '00D4BB',
               '2C99A8', '00C2FF', '344593', '6473FF', '0018EC', '8438FF', '520085', 'CB38FF', 'FF95C8', 'FF37C7')
        self.palette = [self.hex2rgb('#' + c) for c in hex]
        self.n = len(self.palette)

    def __call__(self, i, bgr=False):
        c = self.palette[int(i) % self.n]
        return (c[2], c[1], c[0]) if bgr else c

    @staticmethod
    def hex2rgb(h):  # rgb order (PIL)
        return tuple(int(h[1 + i:1 + i + 2], 16) for i in (0, 2, 4))

##### Method 1 to draw bboxes and save them according to class

In [ ]:
def draw_all_bboxes_classwise(generated_txts, save_dir, label, colors):

    for i in tqdm(range(len(generated_txts))):
        filename = generated_txts[i]

        if not filename.endswith("classes.txt"):
            txt = filename.split("/")[-1]
            img = txt.replace(".txt", ".jpg")
            image = os.path.join(images_path, img)
            im = cv2.imread(image)
            h, w = im.shape[:2]
            count = 0
            with open(filename, 'r') as f:
                lines = f.readlines()
                for line in lines:
                    line = line.split(" ")
                    cls = int(line[0])
                    if cls == label:
                        count += 1
                    classname = labels_names[cls]
                    x1,y1,w1,h1 = map(float, line[1:])
                    xmin = int((x1*w) - (w1*w)/2.0)
                    ymin = int((y1*h) - (h1*h)/2.0)
                    xmax = int((x1*w) + (w1*w)/2.0)
                    ymax = int((y1*h) + (h1*h)/2.0)
                    color = colors(cls, True)
                    thickness = 10
                    im = cv2.rectangle(im, (xmin, ymin), (xmax, ymax), color, thickness)
                    im = cv2.putText(im, classname, (xmin-10, ymin-10), cv2.FONT_HERSHEY_SIMPLEX, 2, color, thickness, cv2.LINE_AA)
            if count:                  
                cv2.imwrite(os.path.join(save_dir, str(label), img), im)   

#### Method 2 : draw all bboxes at a time and save the images

In [ ]:
def draw_all_bboxes(generated_txts, save_dir, colors):

    for i in tqdm(range(len(generated_txts))):
        filename = generated_txts[i]

        if not filename.endswith("classes.txt"):
            txt = filename.split("/")[-1]
            img = txt.replace(".txt", ".jpg")
            image = os.path.join(images_path, img)
            im = cv2.imread(image)
            h, w = im.shape[:2]
            with open(filename, 'r') as f:
                lines = f.readlines()
                for line in lines:
                    line = line.split(" ")
                    cls = int(line[0])
                    classname = labels_names[cls]
                    x1,y1,w1,h1 = map(float, line[1:])
                    xmin = int((x1*w) - (w1*w)/2.0)
                    ymin = int((y1*h) - (h1*h)/2.0)
                    xmax = int((x1*w) + (w1*w)/2.0)
                    ymax = int((y1*h) + (h1*h)/2.0)
                    color = colors(cls, True)
                    thickness = 10
                    im = cv2.rectangle(im, (xmin, ymin), (xmax, ymax), color, thickness)
                    im = cv2.putText(im, classname, (xmin-10, ymin-10), cv2.FONT_HERSHEY_SIMPLEX, 2, color, thickness, cv2.LINE_AA)
               
            cv2.imwrite(os.path.join(save_dir, img), im) 

#### Main function

In [ ]:
def Visualize_yolo_bboxes_and_Write(images_path, labels_path, save_dir, labels_names, count, classwise = False):

    # Check whether paths exists or not
    if not os.path.isdir(images_path):
        print("Images path does not exists")
    if not os.path.isdir(labels_path):
        print("Labels path does not exists")
    # Create folder to save new images
    save_dir = os.path.join(save_dir, "visualization")
    if not os.path.isdir(save_dir):
        os.makedirs(save_dir)
    
    # list out all the labels
    generated_txts = glob.glob(labels_path + "/" + "*.txt")[:count]
    # colors class
    colors = Colors()

    if classwise:
        # create classwise folder and draw bboxes on image and write 
        for label in labels_names:
            label = labels_names.index(label)
            

            if not os.path.isdir(os.path.join(save_dir, str(label))):
                os.makedirs(os.path.join(save_dir, str(label)))

            draw_all_bboxes_classwise(generated_txts, save_dir, label, colors)
    else:
        # create 1 folder , draw all classes bboxes and write the image
        draw_all_bboxes(generated_txts, save_dir, colors)

    print("Done") 
    print(f"Saved Results at {save_dir}")

In [ ]:
            
count = -1
# Arpitha, kavitha, Nischitha
labels_names = ['car', 'bus', 'None', 'zeep', 'tractor']

# declare paths
images_path = "../../Documentation/Images" 
labels_path = "../../Documentation/labels"
save_dir = '../../Documentation/'

# classwise or not
classwise = True
# Start verifying
Visualize_yolo_bboxes_and_Write(images_path, labels_path, save_dir, labels_names, count, classwise)

100%|██████████| 30/30 [00:04<00:00,  7.03it/s]

Done
Saved Results at ../../Documentation/visualization
